# SageMaker Tensorflow 컨테이너를 활용한 하이퍼 파라미터 튜닝

이 학습서는 **SageMaker TensorFlow 컨테이너**를 사용하여 디바이스 장애 데이터 세트를 학습하기 위해 컨볼 루션 신경망 모델을 작성하는 방법에 중점을 둡니다. 하이퍼 파라미터 튜닝을 활용하여 다양한 하이퍼 파라미터 조합으로 여러 교육 작업을 시작하여 최상의 모델 교육 결과를 제공합니다.

## 환경 설정
워크 플로를 시작하기 전에 몇 가지 사항을 설정하겠습니다.

1. 학습 데이터 세트 및 모델 아티팩트가 저장 될 s3 버킷을 지정합니다.
2. s3 버킷과 같은 리소스에 액세스하기 위해 sagemaker에 전달 될 실행 역할을 얻습니다.

In [1]:
import sagemaker

bucket = 'chris.loves.ai' # we are using a default bucket here but you can change it to any bucket in your account

role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example

이제 필요한 Python 라이브러리를 가져옵니다.

In [2]:
import boto3
from time import gmtime, strftime
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

## Load dataset

In [16]:
import tensorflow as tf

## 데이터 업로드
``sagemaker.Session.upload_data`` 함수를 사용하여 데이터 셋을 S3 위치에 업로드합니다. 반환 값은 위치를 식별합니다. 나중에 훈련 작업을 시작할 때이 값을 사용합니다.

In [6]:
# inputs = sagemaker.Session().upload_data(path='data', bucket=bucket, key_prefix='/dataset/device-failure')
# print (inputs)

s3://sagemaker-us-east-2-318688069356/sagemaker/DEMO-hpo-tensorflow-high/data/mnist


In [76]:
!curl https://raw.githubusercontent.com/chris-chris/sagemaker-mlops/master/tf2.py > tf2.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3341  100  3341    0     0  11211      0 --:--:-- --:--:-- --:--:-- 11173


## 분산 훈련을위한 스크립트 작성

네트워크 모델의 전체 코드는 다음과 같습니다.

In [77]:
# TensorFlow 2.1 script
!pygmentize 'tf2.py'

"""Tensorflow2 Keras Version Model Training"""

import argparse
import os
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.optimizers import Adam

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

def model(train_dataset, test_dataset, args):
  """Generate a simple model"""
  model = Sequential(
      [
        Dense(args.fc1, input_shape=(8,)),
        Dense(args.fc2, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(args.dropout),
        Dense(1, activation='sigmoid')
      ])
  # eval_callback = LambdaCallback(
  #     on_epoch_end=lambda epoch, logs: logs.update(
  #         {'mean_logits': K.eval(mean)}
  #     ))
  model.compile(optimizer=Adam(learning_rate=args.learning_rate), loss='binary_crossentropy')
  early_stopping = EarlyStopping(monit

## 하이퍼 파라미터 튜닝 작업 설정
* 아래의 기본 설정에서 하이퍼 파라미터 튜닝 작업을 완료하는 데 약 30 분이 걸릴 수 있습니다. *

이제 다음 단계에 따라 SageMaker Python SDK를 사용하여 하이퍼 파라미터 튜닝 작업을 설정합니다.

* TensorFlow 교육 작업을 설정하기위한 추정기를 작성하십시오.
* 튜닝하려는 하이퍼 파라미터의 범위를 정의합니다.이 예에서는 "learning_rate"를 튜닝합니다.
* 튜닝 작업을위한 객관적인 메트릭 정의
* 위의 설정과 튜닝 리소스 구성으로 하이퍼 파라미터 튜너를 만듭니다.

SageMaker에서 단일 TensorFlow 작업을 학습하는 것과 유사하게 TensorFlow 스크립트, IAM 역할 및 (작업 별) 하드웨어 구성을 전달하는 TensorFlow 추정기를 정의합니다.

In [78]:
device_failure_estimator = TensorFlow(entry_point='tf2.py',
                             role=role,
                             train_instance_count=1,
                             train_instance_type='ml.p2.xlarge',
                             framework_version='2.1.0',
                             py_version='py3',
                             distributions={'parameter_server': {'enabled': True}})

Estimator를 정의한 후에는 조정하려는 하이퍼 파라미터와 가능한 값을 지정할 수 있습니다. 하이퍼 파라미터에는 세 가지 유형이 있습니다.
-범주 형 매개 변수는 이산 세트에서 하나의 값을 가져와야합니다. 가능한 값 목록을`CategoricalParameter (list)`에 전달하여이를 정의합니다.
-연속 매개 변수는`ContinuousParameter (min, max)`에 의해 정의 된 최소값과 최대 값 사이의 임의의 실수 값을 취할 수 있습니다.
-정수 매개 변수는`IntegerParameter (min, max)`에 의해 정의 된 최소값과 최대 값 사이의 정수 값을 가질 수 있습니다

* 가능하면 값을 최소 제한 유형으로 지정하는 것이 가장 좋습니다. 예를 들어 학습 속도를 0.01과 0.2 사이의 연속 값으로 조정하면 값이 0.01, 0.1, 0.15 또는 0.2 인 범주 형 매개 변수로 조정하는 것보다 더 나은 결과를 얻을 수 있습니다. *

In [79]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.01, 0.2),
    'dropout': ContinuousParameter(0.01, 0.5),
    'fc1': CategoricalParameter([32,64,128,256,512]),
    'fc2': CategoricalParameter([32,64,128,256]),
}

다음으로 조정하려는 객관적인 지표와 그 정의를 지정합니다. 여기에는 훈련 작업의 CloudWatch 로그에서 해당 지표를 추출하는 데 필요한 정규식 (Regex)이 포함됩니다. 이 특별한 경우 스크립트에서 손실 값을 생성하고이를 객관적인 메트릭으로 사용하고 objective_type을 '최소화'로 설정하여 하이퍼 파라미터 튜닝이 최상의 하이퍼 파라미터 설정을 검색 할 때 객관적인 메트릭을 최소화하도록합니다. 기본적으로 objective_type은 'maximize'로 설정되어 있습니다.

In [80]:
objective_metric_name = 'test_loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'test_loss',
                       'Regex': 'test_loss: ([0-9\\.]+)'}]

이제 우리는 전달할`HyperparameterTuner` 객체를 만들 것입니다 :

- 위에서 만든 TensorFlow Estimator
- 하이퍼 파라미터 범위
- 객관적인 메트릭 이름 및 정의
- 실행할 총 교육 작업 수 및 병렬로 실행할 수있는 교육 작업 수와 같은 튜닝 리소스 구성

In [83]:
tuner = HyperparameterTuner(device_failure_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=3,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

## 하이퍼 파라미터 튜닝 작업 시작

마지막으로`.fit ()`을 호출하고 S3 경로를 기차 및 테스트 데이터 세트로 전달하여 하이퍼 프라 미터 튜닝 작업을 시작할 수 있습니다.

하이퍼 파라미터 튜닝 작업이 생성 된 후 다음 단계에서 진행 상황을 확인하기 위해 튜닝 작업을 설명 할 수 있어야하며 SageMaker 콘솔-> 작업으로 이동하여 하이퍼 파라미터 튜닝 작업의 진행 상황을 확인할 수 있습니다.

In [84]:
training_data_uri = 's3://chris.loves.ai/dataset/device-failure'
tuner.fit(training_data_uri)

하이퍼 파라미터 튜닝 작업 상태를 빠르게 확인하여 성공적으로 시작되었는지 확인하십시오.

In [85]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

u'InProgress'

## 튜닝 작업 결과 분석-튜닝 작업이 완료된 후

튜닝 작업 결과를 분석하는 예제 코드를 보려면 SageMaker 웹 콘솔을 참조하십시오.

## 최고의 모델 배포

최고의 모델을 얻었으므로 이제 엔드 포인트에 배포 할 수 있습니다. 모델을 배포하는 방법을 보려면 다른 SageMaker 샘플 노트북 또는 SageMaker 설명서를 참조하십시오.